In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import sys

W0709 12:27:58.967900 139995656779584 __init__.py:308] Limited tf.compat.v2.summary API due to missing TensorBoard installation.
W0709 12:27:58.986637 139995656779584 __init__.py:335] Limited tf.summary API due to missing TensorBoard installation.


In [2]:
# Add src directory to path for local imports
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Local imports
from tf_utils import gpu_grow_memory

In [3]:
gpu_grow_memory()

In [4]:
tfds.disable_progress_bar()

In [5]:
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [6]:
mnist_train, mnist_test = datasets['train'], datasets['test']

In [7]:
strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(['/GPU:0', '/GPU:1'])

In [8]:
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 4


In [9]:
# You can also do info.splits.total_num_examples to get the total
# number of examples in the dataset.

num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [10]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [11]:
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [12]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [13]:
# Define the checkpoint directory to store the checkpoints

checkpoint_dir = './checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [14]:
# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
    if epoch < 3:
        return 1e-3
    elif epoch >= 3 and epoch < 7:
        return 1e-4
    else:
        return 1e-5

In [15]:
# Callback for printing the LR at the end of each epoch.
class PrintLR(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print('\nLearning rate for epoch {} is {}'.format(epoch + 1,
                                                          model.optimizer.lr.numpy()))

In [16]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

In [17]:
# model.fit(train_dataset, epochs=12, callbacks=callbacks)
hist = model.fit(train_dataset, epochs=12, callbacks=None)

Train on None steps
Epoch 1/12
234/234 [==============================] - 18s 75ms/step - loss: 0.3608 - accuracy: 0.9003
Epoch 2/12
234/234 [==============================] - 5s 19ms/step - loss: 0.1127 - accuracy: 0.9684
Epoch 3/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0745 - accuracy: 0.9788
Epoch 4/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0540 - accuracy: 0.9847
Epoch 5/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0428 - accuracy: 0.9878
Epoch 6/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0369 - accuracy: 0.9891
Epoch 7/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0300 - accuracy: 0.9914
Epoch 8/12
234/234 [==============================] - 5s 19ms/step - loss: 0.0251 - accuracy: 0.9930
Epoch 9/12
234/234 [==============================] - 4s 19ms/step - loss: 0.0203 - accuracy: 0.9943
Epoch 10/12
234/234 [==============================] - 5s 20ms/step - 

In [18]:
model.evaluate(train_dataset)

    234/Unknown - 5s 22ms/step - loss: 0.0083 - accuracy: 0.9985

[0.008293194457257772, 0.9984809]